In [ ]:
# run this line to update the covid data file
import urllib.request as url
url.urlretrieve('http://cowid.netlify.com/data/full_data.csv', 'data/covid19.csv')

In [ ]:
import geopandas as gpd
import pandas as pd
world = gpd.read_file('data/110m_countries.shp')[['ADMIN', 'ADM0_A3', 'geometry']]
world = world.drop(world.index[159]) # remove Antarctica
world.columns = ['location', 'code', 'shape'] # rename columns
cdata = pd.read_csv('data/covid19.csv').fillna(0)